# Evaluation Functions

In [7]:
import argparse
import json

from conll16st.confusion_matrix import ConfusionMatrix, Alphabet
from conll16st.conn_head_mapper import ConnHeadMapper
import conll16st.validator as validator

import itertools
import numpy as np
import matplotlib.pyplot as plt

from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

CONN_HEAD_MAPPER = ConnHeadMapper()

def evaluate(gold_list, predicted_list):
    connective_cm = evaluate_connectives(gold_list, predicted_list)
    arg1_cm, arg2_cm, rel_arg_cm = evaluate_argument_extractor(gold_list, predicted_list)
    sense_cm = evaluate_sense(gold_list, predicted_list)

    print 'Explicit connectives         : Precision %1.4f Recall %1.4f F1 %1.4f' % connective_cm.get_prf('yes')
    print 'Arg 1 extractor              : Precision %1.4f Recall %1.4f F1 %1.4f' % arg1_cm.get_prf('yes')
    print 'Arg 2 extractor              : Precision %1.4f Recall %1.4f F1 %1.4f' % arg2_cm.get_prf('yes')
    print 'Arg1 Arg2 extractor combined : Precision %1.4f Recall %1.4f F1 %1.4f' % rel_arg_cm.get_prf('yes')
    print 'Sense classification--------------'
    sense_cm.print_summary()
    print 'Overall parser performance --------------'
    precision, recall, f1 = sense_cm.compute_micro_average_f1()
    print 'Precision %1.4f Recall %1.4f F1 %1.4f' % (precision, recall, f1)
    return connective_cm, arg1_cm, arg2_cm, rel_arg_cm, sense_cm, precision, recall, f1


def evaluate_argument_extractor(gold_list, predicted_list):
    """Evaluate argument extractor at Arg1, Arg2, and relation level

    """
    gold_arg1 = [(x['DocID'], x['Arg1']['TokenList']) for x in gold_list]
    predicted_arg1 = [(x['DocID'], x['Arg1']['TokenList']) for x in predicted_list]
    arg1_cm = compute_binary_eval_metric(gold_arg1, predicted_arg1, span_exact_matching)

    gold_arg2 = [(x['DocID'], x['Arg2']['TokenList']) for x in gold_list]
    predicted_arg2 = [(x['DocID'], x['Arg2']['TokenList']) for x in predicted_list]
    arg2_cm = compute_binary_eval_metric(gold_arg2, predicted_arg2, span_exact_matching)

    gold_arg12 = [(x['DocID'], (x['Arg1']['TokenList'], x['Arg2']['TokenList'])) \
            for x in gold_list]
    predicted_arg12 = [(x['DocID'], (x['Arg1']['TokenList'], x['Arg2']['TokenList'])) \
            for x in predicted_list]
    rel_arg_cm = compute_binary_eval_metric(gold_arg12, predicted_arg12, spans_exact_matching)
    return arg1_cm, arg2_cm, rel_arg_cm

def evaluate_connectives(gold_list, predicted_list):
    """Evaluate connective recognition accuracy for explicit discourse relations

    """
    explicit_gold_list = [(x['DocID'], x['Connective']['TokenList'], x['Connective']['RawText']) \
            for x in gold_list if x['Type'] == 'Explicit']
    explicit_predicted_list = [(x['DocID'], x['Connective']['TokenList']) \
            for x in predicted_list if x['Type'] == 'Explicit']
    connective_cm = compute_binary_eval_metric(
            explicit_gold_list, explicit_predicted_list, connective_head_matching)    
    return connective_cm

def spans_exact_matching(gold_doc_id_spans, predicted_doc_id_spans):
    """Matching two lists of spans

    Input:
        gold_doc_id_spans : (DocID , a list of lists of tuples of token addresses)
        predicted_doc_id_spans : (DocID , a list of lists of token indices)

    Returns:
        True if the spans match exactly
    """
    exact_match = True
    gold_docID = gold_doc_id_spans[0]
    gold_spans = gold_doc_id_spans[1]
    predicted_docID = predicted_doc_id_spans[0]
    predicted_spans = predicted_doc_id_spans[1]

    for gold_span, predicted_span in zip(gold_spans, predicted_spans):
        exact_match = span_exact_matching((gold_docID,gold_span), (predicted_docID, predicted_span)) \
                and exact_match
    return exact_match

def span_exact_matching(gold_span, predicted_span):
    """Matching two spans

    Input:
        gold_span : a list of tuples :(DocID, list of tuples of token addresses)
        predicted_span : a list of tuples :(DocID, list of token indices)

    Returns:
        True if the spans match exactly
    """
    gold_docID = gold_span[0]
    predicted_docID = predicted_span[0]
    if gold_docID != predicted_docID:
        return False
    gold_token_indices = [x[2] for x in gold_span[1]]
    predicted_token_indices = predicted_span[1]
    return gold_docID == predicted_docID and gold_token_indices == predicted_token_indices

def connective_head_matching(gold_raw_connective, predicted_raw_connective):
    """Matching connectives

    Input:
        gold_raw_connective : (DocID, a list of tuples of token addresses, raw connective token)
        predicted_raw_connective : (DocID, a list of tuples of token addresses)

    A predicted raw connective is considered iff
        1) the predicted raw connective includes the connective "head"
        2) the predicted raw connective tokens are the subset of predicted raw connective tokens

    For example:
        connective_head_matching('two weeks after', 'weeks after')  --> True
        connective_head_matching('two weeks after', 'two weeks')  --> False not covering head
        connective_head_matching('just because', 'because')  --> True
        connective_head_matching('just because', 'simply because')  --> False not subset
        connective_head_matching('just because', 'since')  --> False
    """
    gold_docID, gold_token_address_list, gold_tokens = gold_raw_connective
    predicted_docID, predicted_token_list = predicted_raw_connective
    if gold_docID != predicted_docID:
        return False

    gold_token_indices = [x[2] for x in gold_token_address_list]

    if gold_token_address_list == predicted_token_list:
        return True
    elif not set(predicted_token_list).issubset(set(gold_token_indices)):
        return False
    else:
        conn_head, indices = CONN_HEAD_MAPPER.map_raw_connective(gold_tokens)
        gold_head_connective_indices = [gold_token_indices[x] for x in indices]
        return set(gold_head_connective_indices).issubset(set(predicted_token_list))

def evaluate_sense(gold_list, predicted_list):
    """Evaluate sense classifier

    The label ConfusionMatrix.NEGATIVE_CLASS is for the relations 
    that are missed by the system
    because the arguments don't match any of the gold relations.
    """
    sense_alphabet = Alphabet()
    valid_senses = validator.identify_valid_senses(gold_list)
    for relation in gold_list:
        sense = relation['Sense'][0]
        if sense in valid_senses:
            sense_alphabet.add(sense)

    sense_alphabet.add(ConfusionMatrix.NEGATIVE_CLASS)

    sense_cm = ConfusionMatrix(sense_alphabet)
    gold_to_predicted_map, predicted_to_gold_map = \
            _link_gold_predicted(gold_list, predicted_list, spans_exact_matching)

    for i, gold_relation in enumerate(gold_list):
        gold_sense = gold_relation['Sense'][0]
        if gold_sense in valid_senses:
            if i in gold_to_predicted_map:
                predicted_sense = gold_to_predicted_map[i]['Sense'][0]
                if predicted_sense in gold_relation['Sense']:
                    sense_cm.add(predicted_sense, predicted_sense)
                else:
                    if not sense_cm.alphabet.has_label(predicted_sense):
                        predicted_sense = ConfusionMatrix.NEGATIVE_CLASS
                    sense_cm.add(predicted_sense, gold_sense)
            else:
                sense_cm.add(ConfusionMatrix.NEGATIVE_CLASS, gold_sense)

    for i, predicted_relation in enumerate(predicted_list):
        if i not in predicted_to_gold_map:
            predicted_sense = predicted_relation['Sense'][0]
            if not sense_cm.alphabet.has_label(predicted_sense):
                predicted_sense = ConfusionMatrix.NEGATIVE_CLASS
            sense_cm.add(predicted_sense, ConfusionMatrix.NEGATIVE_CLASS)
    return sense_cm


def combine_spans(span1, span2):
    """Merge two text span dictionaries

    """
    new_span = {}
    new_span['CharacterSpanList'] = span1['CharacterSpanList'] + span2['CharacterSpanList']
    new_span['SpanList'] = span1['SpanList'] + span2['SpanList']
    new_span['RawText'] = span1['RawText'] + span2['RawText']
    new_span['TokenList'] = span1['TokenList'] + span2['TokenList']
    return new_span

def compute_binary_eval_metric(gold_list, predicted_list, matching_fn):
    """Compute binary evaluation metric

    """
    binary_alphabet = Alphabet()
    binary_alphabet.add('yes')
    binary_alphabet.add('no')
    cm = ConfusionMatrix(binary_alphabet)
    matched_predicted = [False for x in predicted_list]
    for gold_span in gold_list:
        found_match = False
        for i, predicted_span in enumerate(predicted_list):
            if matching_fn(gold_span, predicted_span) and not matched_predicted[i]:
                cm.add('yes', 'yes')
                matched_predicted[i] = True
                found_match = True
                break
        if not found_match:
            cm.add('no', 'yes')
    # Predicted span that does not match with any
    for matched in matched_predicted:
        if not matched:
            cm.add('yes', 'no')
    return cm


def _link_gold_predicted(gold_list, predicted_list, matching_fn):
    """Link gold standard relations to the predicted relations

    A pair of relations are linked when the arg1 and the arg2 match exactly.
    We do this because we want to evaluate sense classification later.

    Returns:
        A tuple of two dictionaries:
        1) mapping from gold relation index to predicted relation index
        2) mapping from predicted relation index to gold relation index
    """
    gold_to_predicted_map = {}
    predicted_to_gold_map = {}
    gold_arg12_list = [(x['DocID'], (x['Arg1']['TokenList'], x['Arg2']['TokenList']))
            for x in gold_list]
    predicted_arg12_list = [(x['DocID'], (x['Arg1']['TokenList'], x['Arg2']['TokenList']))
            for x in predicted_list]
    for gi, gold_span in enumerate(gold_arg12_list):
        for pi, predicted_span in enumerate(predicted_arg12_list):
            if matching_fn(gold_span, predicted_span):
                gold_to_predicted_map[gi] = predicted_list[pi]
                predicted_to_gold_map[pi] = gold_list[gi]
    return gold_to_predicted_map, predicted_to_gold_map


def main(gold,predicted):
    #parser = argparse.ArgumentParser(
    #    description="Evaluate system's output against the gold standard")
    #parser.add_argument('gold', help='Gold standard file')
    #parser.add_argument('predicted', help='System output file')
    #args = parser.parse_args()
    gold_list = [json.loads(x) for x in open(gold)]
    predicted_list = [json.loads(x) for x in open(predicted)]
    print '\n================================================'
    print 'Evaluation for all discourse relations'
    evaluate(gold_list, predicted_list)

    print '\n================================================'
    print 'Evaluation for explicit discourse relations only'
    explicit_gold_list = [x for x in gold_list if x['Type'] == 'Explicit']
    explicit_predicted_list = [x for x in predicted_list if x['Type'] == 'Explicit']
    evaluate(explicit_gold_list, explicit_predicted_list)

    print '\n================================================'
    print 'Evaluation for non-explicit discourse relations only (Implicit, EntRel, AltLex)'
    non_explicit_gold_list = [x for x in gold_list if x['Type'] != 'Explicit']
    non_explicit_predicted_list = [x for x in predicted_list if x['Type'] != 'Explicit']
    evaluate(non_explicit_gold_list, non_explicit_predicted_list)
    
    
    
def plot_confusion_matrix(y_test,y_pred, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """

    cm = confusion_matrix(y_test, y_pred)


    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
    plt.figure(figsize=(18,18))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
    plt.show()


Functionality Evaluation

3 Parts
- Connectives
- Arg1/Arg2 Span
- Sense Validation

only exact matching pairs are True

# Comparison

## Create Unified Formats

In [5]:
import numpy as np
from sklearn.metrics import f1_score
import pandas as pd
from collections import Counter
from itertools import product

In [6]:
gold_example = "data/gold_standard/conll16st-en-03-29-16-blind-test/relations.json"
parser_example = "data/submissions/conll16-st-end-to-end-output/conll16-st-blind-test-en/shallow-discourse-parsing-oslopots-run-2016-05-11-01-58-41.zip"

gold_example_list = [json.loads(x) for x in open(gold_example)]
parser_example_list = [json.loads(x) for x in open(parser_example)]

ValueError: No JSON object could be decoded

In [116]:
[i for i in stepanov_predicted_list if i['Type'] == 'Explicit' and i['DocID'] == 'wsj_2200'][0]

{u'Arg1': {u'CharacterSpanList': [[517, 564]],
  u'RawText': u'to restrict the RTC to Treasury borrowings only',
  u'TokenList': [85, 86, 87, 88, 89, 90, 91, 92]},
 u'Arg2': {u'CharacterSpanList': [[573, 629]],
  u'RawText': u'the agency receives specific congressional authorization',
  u'TokenList': [95, 96, 97, 98, 99, 100]},
 u'Connective': {u'CharacterSpanList': [[566, 572]],
  u'RawText': u'unless',
  u'TokenList': [94]},
 u'DocID': u'wsj_2200',
 u'ID': 35709,
 u'Sense': [u'Expansion.Alternative'],
 u'Type': u'Explicit'}

In [115]:
[i for i in gold_list if i['Type'] == 'Explicit' and i['DocID'] == 'wsj_2200'][0]

{u'Arg1': {u'CharacterSpanList': [[517, 564]],
  u'RawText': u'to restrict the RTC to Treasury borrowings only',
  u'TokenList': [[517, 519, 85, 2, 3],
   [520, 528, 86, 2, 4],
   [529, 532, 87, 2, 5],
   [533, 536, 88, 2, 6],
   [537, 539, 89, 2, 7],
   [540, 548, 90, 2, 8],
   [549, 559, 91, 2, 9],
   [560, 564, 92, 2, 10]]},
 u'Arg2': {u'CharacterSpanList': [[573, 629]],
  u'RawText': u'the agency receives specific congressional authorization',
  u'TokenList': [[573, 576, 95, 2, 13],
   [577, 583, 96, 2, 14],
   [584, 592, 97, 2, 15],
   [593, 601, 98, 2, 16],
   [602, 615, 99, 2, 17],
   [616, 629, 100, 2, 18]]},
 u'Connective': {u'CharacterSpanList': [[566, 572]],
  u'RawText': u'unless',
  u'TokenList': [[566, 572, 94, 2, 12]]},
 u'DocID': u'wsj_2200',
 u'ID': 35709,
 u'Sense': [u'Expansion.Alternative'],
 u'Type': u'Explicit'}

Each Relation is defined by:

- DocID
- Arg1 TokenSpan
- Arg2 TokenSpan
- Type
- Connective (Explicit, maybe Implicit)
- Sense

In [7]:
def create_comparison_format(relation,gold=True):
    conn = dict(relation["Connective"])
    if conn.has_key("CharacterSpanList"):
        del conn["CharacterSpanList"]
    if not conn.has_key("RawText"):
        conn["RawText"] = ""
    
    dic = {
        "DocID": relation["DocID"],
        "RelID": 0,
        "Sense": relation["Sense"],
        "Type" : relation["Type"],
        "Connective": conn   
    }
    
    if gold:
        dic["Arg1TokenList"] = [token[2] for token in relation["Arg1"]["TokenList"]]
        dic["Arg2TokenList"] = [token[2] for token in relation["Arg2"]["TokenList"]]
        dic["Parser"] = "Gold"
    else:
        dic["Arg1TokenList"] = relation["Arg1"]["TokenList"]
        dic["Arg2TokenList"] = relation["Arg2"]["TokenList"]
        dic["Parser"] = "Pred"
    
    return dic

In [8]:
def write_comparison_format_to_file(relations,file_path):
    with open(file_path, 'w') as f:
        json.dump(relations, f)

In [9]:
def read_comparison_format_from_file(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
        f.close()
    return data

### Create Files

In [10]:
dev_wang2015_file_path = "data/submissions/wang2015_dev/output/Wang2015.json"
dev_wang2015_predicted_list = [json.loads(x) for x in open(dev_wang2015_file_path)]
relations = [create_comparison_format(pred,False) for pred in dev_wang2015_predicted_list]
write_comparison_format_to_file(relations,"data/project_files/dev_wang2105.json")

In [502]:
dev_gold_file_path = "../../conll2016/en.dev/relations.json"
dev_oslopots_file_path = "data/submissions/oslopots_dev/output/output.json"
dev_stepanov_file_path = "data/submissions/stepanov_dev/output/output.json"
dev_ecnucs_file_path = "data/submissions/ecnucs_dev/output/output.json"

In [503]:
#main()
gold_list = [json.loads(x) for x in open(dev_gold_file_path)]
oslopots_predicted_list = [json.loads(x) for x in open(dev_oslopots_file_path)]
stepanov_predicted_list = [json.loads(x) for x in open(dev_stepanov_file_path)]
ecnucs_predicted_list = [json.loads(x) for x in open(dev_ecnucs_file_path)]

In [504]:
#predictions = [("oslopots",oslopots_predicted_list),("stepanov",stepanov_predicted_list),("ecnucs",ecnucs_predicted_list)]
predictions = [("wang2015",)]

In [505]:
gold_relations = [create_comparison_format(gold,True) for gold in gold_list]
write_comparison_format_to_file(gold_relations,"data/project_files/dev_gold_relations.json")

for name,prediction in predictions:
    relations = [create_comparison_format(pred,False) for pred in prediction]
    write_comparison_format_to_file(relations,"data/project_files/dev_"+name+".json")

## Map Relations together

Some of the relations are completely equal (because the evaluation only allows exact matching spans), but some only have parts in common or are completely missed by the parsers.

In [218]:
oslopots_relations = read_comparison_format_from_file("data/project_files/dev_oslopots.json")
gold_relations = read_comparison_format_from_file("data/project_files/dev_gold_relations.json")

In [216]:
def diff_arg(gold_token,pred_token):
    diff = len(
        set(pred_token).intersection(set(gold_token))
    )/len(
        set(pred_token).union(set(gold_token)))
    if diff != 1 and diff != 0:
        print diff
    return diff

In [198]:
def map_gold_pred_rel(gold_list,pred_list):
    pass

In [252]:
docIDs = set([rel["DocID"] for rel in gold_relations])
mapping = []
for docID in docIDs:
    pred_doc_rel = [rel for rel in oslopots_relations if rel['DocID'] == docID]
    gold_doc_rel = [rel for rel in gold_relations if rel['DocID'] == docID]

    for gold_rel in gold_doc_rel:
        gold_arg1 = gold_rel["Arg1TokenList"]
        gold_arg2 = gold_rel["Arg2TokenList"]
        gold_pred_map = []
        for pred_rel in pred_doc_rel:
            pred_arg1 = pred_rel["Arg1TokenList"]
            pred_arg2 = pred_rel["Arg2TokenList"]
            gold_pred_map += [(diff_arg(gold_arg1,pred_arg1)+diff_arg(gold_arg2,pred_arg2))/2]
        best_match_index = np.argmax(gold_pred_map)
        mapping += [
            [gold_rel["RelID"],
            pred_doc_rel[best_match_index]["RelID"]]
        ]

In [254]:
found_gold,found_pred = zip(*mapping)

In [260]:
len(found_gold)/len(gold_relations)

1

### Conclusion

The recognition of the right spans is most of the times right by the examinated parsers. Therefore, it doesn't help to explore span overlapping anymore.

## Combine each relation of Gold and all Predictions

In [279]:
def combine_rel(gold_list,parser_pred):
    combinations = {gold["RelID"]:[gold] for gold in gold_list}
    for name,prediction in parser_pred:
        for pred in prediction:
            pred["Parser"] = name
            combinations[pred["RelID"]] += [pred]
    return combinations

In [489]:
def compare_attr(parse_pairs,attr,listing=True):
    pair_values = parse_pairs.values()
    rows = []
    for pair in pair_values:
        if listing:
            gold_sense = pair[0][attr][0]
            all_senses = [gold_sense] + [parse[attr][0] for parse in pair[1:]]
        else:
            gold_sense = pair[0][attr]
            all_senses = [gold_sense] + [parse[attr] for parse in pair[1:]]
        rows += [tuple(all_senses)]
    
    return rows

In [550]:
def sense_statistics(sense_comparison):
    zip_sense_comparison = zip(*sense_comparison)
    gold_senses = zip_sense_comparison[0]
    #Count all combination of senses
    sense_counter = Counter(sense_comparison)
    #Different Senses
    set_senses = set(gold_senses)
    #How many parsers will be compared
    len_parser = len(zip_sense_comparison)-1
    sense_rows = []

    for sense in set_senses:
        tmp_senses = set_senses.copy()
        tmp_senses.remove(sense)
        possible_comb = [[sense]] + [tmp_senses for i in range(len_parser)]
        diff_preds = product(*possible_comb)

        equal_correct_parsing = sense_counter[tuple([sense]+[sense]*len_parser)]

        all_wrong_parsing = sum([
            sense_counter[tuple(diff_pred)] 
            for diff_pred in diff_preds])

        total_act_sense_count = equal_correct_parsing + sum([1 for i in gold_senses if i == sense])


        equal_wrong_parsing = sum([
            sense_counter[tuple([sense]+[other_sense]*len_parser)] 
            for other_sense in tmp_senses])
        
        parser_columns = []

        #Parser is better than all the other
        parser_better = []
        for index_parser in range(1,len_parser+1):
            tmp_possible_comb = possible_comb[:]
            tmp_possible_comb[index_parser] = [sense]
            other_diff_preds = product(*tmp_possible_comb)
            
            pars_better = sum([sense_counter[other_diff_pred] for other_diff_pred in other_diff_preds])
            total_pred_sense_count = Counter(zip_sense_comparison[index_parser])[sense]#sum([1 for i in zip_sense_comparison[index_parser] if i == sense])

            parser_better += [pars_better,total_pred_sense_count]
            
            column_name = parser_names[index_parser-1]
            parser_columns += ["Only {} right".format(column_name),"Total Pred ({})".format(column_name)]
            
        #Two Parser share opinions
        for parser1,parser2 in itertools.combinations([i for i in range(1,len_parser+1)],2):
            tmp_possible_comb = possible_comb[:]
            tmp_possible_comb[parser1] = [sense]
            tmp_possible_comb[parser2] = [sense]
            
            
            pars_better = sum([sense_counter[comb] for comb in product(*tmp_possible_comb)])
            total_pred_sense_count = sum(
                [1 for pars1_pred,pars2_pred in zip(*[
                    zip_sense_comparison[parser1],
                    zip_sense_comparison[parser2]
                ])
                if (pars2_pred == sense) != (pars1_pred == sense)])
            
            parser_better += [pars_better,total_pred_sense_count]
            
            column_name = " and ".join([parser_names[parser1-1],parser_names[parser2-1]])
            parser_columns += ["{} right".format(column_name),"Total Pred ({})".format(column_name)]
        

        #At least one parser is correct
        correct_sense_poss = [[sense]]+[set_senses]*len_parser
        combination_correct = sum(
            [sense_counter[at_least_one_right] 
             for at_least_one_right in product(*correct_sense_poss) if sense in at_least_one_right[1:]])

        sense_rows += [[sense,equal_correct_parsing,all_wrong_parsing,total_act_sense_count,equal_wrong_parsing,combination_correct]+parser_better]


    return pd.DataFrame(
        columns=["Sense",
                 "Equal Correct",
                 "All Wrong",
                 "Total Act",
                 "Equal Wrong",
                 "At least one correct"]+parser_columns,
        data=sense_rows)


In [506]:
oslopots_relations = read_comparison_format_from_file("data/project_files/dev_oslopots.json")
stepanov_relations = read_comparison_format_from_file("data/project_files/dev_stepanov.json")
ecnucs_relations = read_comparison_format_from_file("data/project_files/dev_ecnucs.json")
gold_relations = read_comparison_format_from_file("data/project_files/dev_gold_relations.json")

In [534]:
parser_pred = [("oslopots",oslopots_relations),("stepanov",stepanov_relations),("ecnucs",ecnucs_relations)]
parser_names = [name for name,pred in parser_pred]

In [535]:
comb = combine_rel(gold_relations,parser_pred)
sense_comparison = compare_attr(comb,"Sense")
zip_sense_comparison = zip(*sense_comparison)

In [516]:
gold_senses = zip_sense_comparison[0]
parser_senses = zip_sense_comparison[1:]
for parser_sense in parser_senses:
    cm = confusion_matrix(gold_senses,parser_sense)
    print(cm)
    print("\n\n")

[[  0   1  19   0   0   0   0   0   0   0   1   0   0   0   0   0   0]
 [  0   5   6   0   0   1   2   0   0   0   1   0   2   0   0   0   0]
 [  0   6 164   6   3   0  36   0   0   0  22   0   7   0   0   0   0]
 [  0   0   0  53   1   0  32   0   0   0   4   0  19   0   1   1   5]
 [  0   0   2  13  20   0  21   0   0   0   7   0   9   0   0   0   0]
 [  0   0   0   0   0  43   0   0   0   0   0   0   0   0   1   0   6]
 [  0   0   1   4   1   0 188   0   0   0  12   1   8   0   0   0   0]
 [  0   0   0   1   1   0   1   0   0   0   2   0   1   0   0   0   0]
 [  0   0   0   0   0   0   0   0   6   0   0   0   0   0   0   0   0]
 [  0   0   1   1   0   0   0   0   0   5   1   0   0   0   0   0   0]
 [  0   0   8   6   1   0  46   0   1   0 231   1  14   0   0   0   0]
 [  0   0   1   3   1   0  26   0   0   0   3  13  10   0   0   0   0]
 [  0   0   1   9   0   0  45   0   0   0  13   1  38   0   1   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   1]
 [  0 

In [533]:
sense_statistics(sense_comparison)

,Sense,Equal Correct,All Wrong,Total Act,Equal Wrong,At least one correct,Only oslopots right,Total Pred (oslopots),Only stepanov right,Total Pred (stepanov)
0,Comparison.Concession,2,12,19,5,5,3,12,0,20
1,Comparison,0,21,21,13,0,0,0,0,0
2,Expansion.Conjunction,158,47,466,20,261,73,308,30,415
3,Contingency.Condition,33,6,83,5,44,10,45,1,52
4,Expansion.Alternative,4,0,10,0,6,2,7,0,5
5,EntRel,112,12,327,2,203,76,420,15,310
6,Contingency.Cause.Result,16,50,88,11,22,4,28,2,46
7,Contingency.Cause.Reason,26,60,142,14,56,27,108,3,72
8,Expansion,0,6,6,2,0,0,0,0,0
9,Temporal,0,1,1,0,0,0,0,0,0


In [551]:
sense_statistics(sense_comparison)

,Sense,Equal Correct,All Wrong,Total Act,Equal Wrong,At least one correct,Only oslopots right,Total Pred (oslopots),Only stepanov right,Total Pred (stepanov),Only ecnucs right,Total Pred (ecnucs),oslopots and stepanov right,Total Pred (oslopots and stepanov),oslopots and ecnucs right,Total Pred (oslopots and ecnucs),stepanov and ecnucs right,Total Pred (stepanov and ecnucs)
0,Comparison.Concession,2,11,19,4,6,0,12,0,20,1,11,0,26,3,3,0,25
1,Comparison,0,21,21,13,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Expansion.Conjunction,152,38,460,9,270,9,308,14,415,9,345,6,351,64,157,16,356
3,Contingency.Condition,33,6,83,5,44,0,45,1,52,0,47,0,29,10,4,0,31
4,Expansion.Alternative,4,0,10,0,6,1,7,0,5,0,5,0,4,1,2,0,2
5,EntRel,95,11,310,1,204,30,420,8,310,1,249,17,330,46,243,7,259
6,Contingency.Cause.Result,16,41,88,4,31,2,28,2,46,9,48,0,40,2,36,0,60
7,Contingency.Cause.Reason,25,39,141,6,77,16,108,2,72,21,97,1,122,11,101,1,115
8,Expansion,0,6,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,Temporal,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [490]:
type_comparison = compare_attr(comb,"Type",False)

In [493]:
sense_statistics(type_comparison)

,Sense,Equal Correct,All Wrong,Total Act,Equal Wrong,At least one correct,Only oslopots right,Total Pred (oslopots),Only stepanov right,Total Pred (stepanov)
0,EntRel,0,215,215,215,0,0,0,0,0
1,Explicit,680,0,1360,0,680,0,680,0,699
2,Implicit,522,0,1044,0,522,0,737,0,737
3,AltLex,0,0,19,0,19,19,19,0,0


In [495]:
Counter(type_comparison)

Counter({(u'AltLex', u'AltLex', u'Explicit'): 19,
         (u'EntRel', u'Implicit', u'Implicit'): 215,
         (u'Explicit', u'Explicit', u'Explicit'): 680,
         (u'Implicit', u'Implicit', u'Implicit'): 522})